In [2]:
# from datetime import datetime
from IPython.display import display, Markdown
from datetime import datetime, date

todays_date = str(datetime.now().date())

display(Markdown(f'# Notebook - Prepare datset for Specificity predictions with BCR-GEX integrated data'))
display(Markdown(f'Author: Lena Erlach'))
display(Markdown(f'Created: 2024-01-21'))
display(Markdown(f'Last modified: {todays_date}'))


import numpy as np
import pandas as pd
import os, sys
UTILS_DIR = '../../src'
sys.path.append(UTILS_DIR)
import utils_nb as utils

# Notebook - Prepare datset for Specificity predictions with BCR-GEX integrated data

Author: Lena Erlach

Created: 2024-01-21

Last modified: 2024-07-18

## Dataprep:
##### 1. Load variable gene expression and load antibody sequence datasets
##### 2. Create datasets of cells having BCR and GEX data modality
a. append 2000 genes, with ~3000 kmer features 

b. reduce dim. with PCA





### Load GEX datasets 

Datsets are top 2000 variable, scaled gene expression values of RBD and OVA dataset separately

In [4]:
# define paths
ROOT_DIR = '../../'

OVA_path_GEX = os.path.join(ROOT_DIR, 'data/raw/scaled_GEX_OVA_harmony_overlappingclonesexcl.csv')
RBD_path_GEX = os.path.join(ROOT_DIR, 'data/raw/scaled_GEX_RBD_harmony_overlappingclonesexcl.csv')
INT_path_GEX = os.path.join(ROOT_DIR, 'data/raw/scaled_GEX_OVA_RBD_int_harmony_overlappingclonesexcl.csv')


OVA_path_VDJ = os.path.join(ROOT_DIR, 'data/processed/processed_OVA_VDJ_aaSeq_df_2023-09-28.csv')
RBD_path_VDJ = os.path.join(ROOT_DIR, 'data/processed/processed_RBD_VDJ_aaSeq_df_2024-01-19.csv')



# OVA_out_comb_feats = os.path.join(ROOT_DIR, f'data/processed/GEX/OVA_BCR_GEX_feats{todays_date}.csv')
# RBD_out_comb_feats =  os.path.join(ROOT_DIR, f'data/processed/GEX/RBD_BCR_GEX_feats{todays_date}.csv')
# INT_out_comb_feats =  os.path.join(ROOT_DIR, f'data/processed/GEX/OVA_RBD_int_BCR_GEX_feats{todays_date}.csv')



OVA DATASET

In [5]:
# Load datasets 
GEX_raw = pd.read_csv(OVA_path_GEX)
GEX_raw.rename(columns={'Unnamed: 0': 'gene_id'}, inplace=True)
GEX_raw.columns = GEX_raw.columns.str.lstrip('_')

VDJ = pd.read_csv(OVA_path_VDJ)



# check which barcodes are intersecting
bcs_GEX = GEX_raw.columns[1:]
bcs_VDJ = VDJ.barcode
bcs_inters = np.intersect1d(bcs_GEX, bcs_VDJ)
print(f'Total num cells - GEX: {len(bcs_GEX)}')
print(f'Total num cells - VDJ: {len(bcs_VDJ)}; specific: {len(VDJ.barcode[VDJ.group_id == 1])}, nonspec: {len(VDJ.barcode[VDJ.group_id == 2])}')

# subset the dataframes and transrom GEX dataframe
GEX_inters = GEX_raw.loc[:, bcs_inters].T
GEX_inters.columns = GEX_raw.gene_id
GEX_inters.rename(columns={'gene_id': 'barcode'}, inplace=True)
GEX_inters.reset_index(drop=True, inplace=True)

VDJ_inters = VDJ[VDJ.barcode.isin(bcs_inters)].reset_index(drop=True)
print(f'Total num cells - inters: {len(VDJ_inters)}; specific: {len(VDJ_inters.barcode[VDJ_inters.group_id == 1])}, nonspec: {len(VDJ_inters.barcode[VDJ_inters.group_id == 2])}')


#### Calculate kmer frequencies
# Calculate the kmer vectors
k=3
seqs = VDJ_inters.VDJ_VJ_aaSeq
all_kmers = utils.generate_all_kmers(seqs, k)
vectors = [utils.freqs_to_vector(utils.kmer_frequencies(seq, k), all_kmers) for seq in seqs]
kmer_arr = np.array(vectors)

# add kmer_arr to VDJ_inters
VDJ_inters_kmer = pd.concat([VDJ_inters.loc[:, ['barcode','sample_id', 'group_id', 'VDJ_VJ_aaSeq', 'VDJ_aaSeq']], pd.DataFrame(kmer_arr)],axis=1)

### Create new dataframe with kmer freqs and gene expression data
GEX_BCR_df =  pd.concat([VDJ_inters_kmer, GEX_inters], axis=1)

GEX_BCR_df.to_csv(OVA_out_comb_feats)

Total num cells - GEX: 8640
Total num cells - VDJ: 3807; specific: 574, nonspec: 3233
Total num cells - inters: 3308; specific: 371, nonspec: 2937


NameError: name 'OVA_out_comb_feats' is not defined

RUN FOR RBD DATASET

In [6]:
# Load datasets 
GEX_raw = pd.read_csv(RBD_path_GEX)
GEX_raw.rename(columns={'Unnamed: 0': 'gene_id'}, inplace=True)
GEX_raw.columns = GEX_raw.columns.str.lstrip('_')

VDJ = pd.read_csv(RBD_path_VDJ)



# check which barcodes are intersecting
bcs_GEX = GEX_raw.columns[1:]
bcs_VDJ = VDJ.barcode
bcs_inters = np.intersect1d(bcs_GEX, bcs_VDJ)
print(f'Total num cells - GEX: {len(bcs_GEX)}')
print(f'Total num cells - VDJ: {len(bcs_VDJ)}; specific: {len(VDJ.barcode[VDJ.group_id == 1])}, nonspec: {len(VDJ.barcode[VDJ.group_id == 2])}')

# subset the dataframes and transrom GEX dataframe
GEX_inters = GEX_raw.loc[:, bcs_inters].T
GEX_inters.columns = GEX_raw.gene_id
GEX_inters.rename(columns={'gene_id': 'barcode'}, inplace=True)
GEX_inters.reset_index(drop=True, inplace=True)

VDJ_inters = VDJ[VDJ.barcode.isin(bcs_inters)].reset_index(drop=True)
print(f'Total num cells - inters: {len(VDJ_inters)}; specific: {len(VDJ_inters.barcode[VDJ_inters.group_id == 1])}, nonspec: {len(VDJ_inters.barcode[VDJ_inters.group_id == 2])}')


#### Calculate kmer frequencies
# Calculate the kmer vectors
k=3
seqs = VDJ_inters.VDJ_VJ_aaSeq
all_kmers = utils.generate_all_kmers(seqs, k)
vectors = [utils.freqs_to_vector(utils.kmer_frequencies(seq, k), all_kmers) for seq in seqs]
kmer_arr = np.array(vectors)

# add kmer_arr to VDJ_inters
VDJ_inters_kmer = pd.concat([VDJ_inters.loc[:, ['barcode','sample_id', 'group_id', 'VDJ_VJ_aaSeq', 'VDJ_aaSeq']], pd.DataFrame(kmer_arr)],axis=1)

### Create new dataframe with kmer freqs and gene expression data
GEX_BCR_df =  pd.concat([VDJ_inters_kmer, GEX_inters], axis=1)

GEX_BCR_df.to_csv(RBD_out_comb_feats)

Total num cells - GEX: 5840
Total num cells - VDJ: 3784; specific: 663, nonspec: 3121
Total num cells - inters: 2561; specific: 510, nonspec: 2051


NameError: name 'RBD_out_comb_feats' is not defined

INT DATASET

In [42]:
# Load datasets 
GEX_raw = pd.read_csv(INT_path_GEX)
GEX_raw.rename(columns={'Unnamed: 0': 'gene_id'}, inplace=True)
GEX_raw.columns = GEX_raw.columns.str.lstrip('_')
# drop duplicated barcodes (only ~40 or so)
GEX_raw = GEX_raw.loc[:,~GEX_raw.columns.duplicated()].copy()


VDJ_O = pd.read_csv(OVA_path_VDJ)
VDJ_O['sample_barcode'] = [f'OVA_{b}' for b in VDJ_O.barcode]
VDJ_R = pd.read_csv(RBD_path_VDJ)
VDJ_R['sample_barcode'] = [f'RBD_{b}' for b in VDJ_R.barcode]

VDJ = pd.concat([VDJ_O, VDJ_R])                         # ---------> maybe barcodes are detected in VDJ and then OVA and RBD??

# drop duplicated barcodes
VDJ = VDJ.loc[~VDJ.sample_barcode.duplicated(), :].copy()


# check which barcodes are intersecting
bcs_GEX = GEX_raw.columns[1:]
bcs_VDJ = VDJ.barcode
bcs_inters = np.intersect1d(bcs_GEX, bcs_VDJ)
print(f'Total num cells - GEX: {len(bcs_GEX)}')
print(f'Total num cells - VDJ: {len(bcs_VDJ)}; specific: {len(VDJ.barcode[VDJ.group_id == 1])}, nonspec: {len(VDJ.barcode[VDJ.group_id == 2])}')

# subset the dataframes and transrom GEX dataframe
GEX_inters = GEX_raw.loc[:, bcs_inters].T
GEX_inters.columns = GEX_raw.gene_id
GEX_inters.rename(columns={'gene_id': 'barcode'}, inplace=True)
GEX_inters.reset_index(drop=True, inplace=True)

VDJ_inters = VDJ[VDJ.barcode.isin(bcs_inters)].reset_index(drop=True)
print(f'Total num cells - inters: {len(VDJ_inters)}; specific: {len(VDJ_inters.barcode[VDJ_inters.group_id == 1])}, nonspec: {len(VDJ_inters.barcode[VDJ_inters.group_id == 2])}')


#### Calculate kmer frequencies
k=3
seqs = VDJ_inters.VDJ_VJ_aaSeq
all_kmers = utils.generate_all_kmers(seqs, k)
vectors = [utils.freqs_to_vector(utils.kmer_frequencies(seq, k), all_kmers) for seq in seqs]
kmer_arr = np.array(vectors)


# add kmer_arr to VDJ_inters
VDJ_inters_kmer = pd.concat([VDJ_inters.loc[:, ['barcode','sample_id', 'group_id', 'VDJ_VJ_aaSeq', 'VDJ_aaSeq']], pd.DataFrame(kmer_arr)],axis=1)

### Create new dataframe with kmer freqs and gene expression data
GEX_BCR_df =  pd.concat([VDJ_inters_kmer, GEX_inters], axis=1)

GEX_BCR_df.to_csv(INT_out_comb_feats)